## this sample script takes continuous variable as outcome, as example

In [1]:
import os
import pandas as pd

import torch
from torch.utils.data import DataLoader

os.chdir('/mnt/f/workspace/pathology/gigapath/code/transfer')
from gliopath.train.task.flex import seed_torch, train, EmbeddingDataset, TaskHead, collate_fn_with_padding, SeriesHead
from gliopath.utils.proces import split_dataset
from gliopath.train.gadget import get_sampler
from gliopath.models.load import giga_slide_enc
from gliopath.models import longn

os.chdir('/mnt/f/workspace/pathology/gigapath')

/home/gyladmin/miniconda3/envs/gigapath/lib/python3.9/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/gyladmin/miniconda3/envs/gigapath/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
seed = 42
dataset_df = pd.read_table('data/metadata.tbl', sep='\t')
test_df = pd.read_table('data/metadata_test.tbl', sep='\t')
embed_path = 'output/tiles/rand_embed/'
z_score = False
outcome_col = ['IDH1','TP53','ATRX','PTEN','EGFR','TERT']
num_classes = len(outcome_col)
batch_size = 4
num_workers = 0
embed_dim = 1536
weighted_sampler = True
feat_layers=[0,1]
num_epochs = 50

unfreeze_schedule = {
    10: [-1],
    20: [-2],
    30: [-3],
}

splits = ['train', 'val', 'test']
split_col = 'split_col'
id_col = 'id'
params = {
    'lr': 0.001,
    'min_lr': 0.0,
    'num_epochs': num_epochs,
    'eval_interval': 10,
    'output_dir': 'output/models/life',
    'optim': 'sgd',
    'weight_decay': 0.01,
    'outcome_type': 'gene',
    'gc_step': 2,
    'freeze_longnet': False,
    'longnet_lr_factor': 0.02,
    'warmup_epochs': 5,
    'active_longnet_layers': None,
    'progressive_unfreeze': True,
    'unfreeze_schedule': unfreeze_schedule,
    'max_frozen_layers': 0,
    'layer_lr_factor': 0.2,
    'min_lr_factor': 0.1,
    'warmup_factor': 0.1,
}

In [3]:
# set the random seed
seed_torch(torch.device('cuda'), 0)
# read the metadata
dataset_df = split_dataset(dataset_df, id_col='id', type_col='tumour_type', val_split=0.2, test_split=0, in_df=True, split_col='split_col')
test_df['split_col'] = 'test'
dataset_df = pd.concat([dataset_df, test_df], ignore_index=True)

# load the dataset
train_dataset, val_dataset, test_dataset = [EmbeddingDataset(dataset_df, embed_path, feat_layer=feat_layers, split_col=split_col, split=split, id_col=id_col, type_col=outcome_col, outcome_type='gene', z_score=z_score) for split in splits]

# set num_classes
print(f'Sample size:\nTrain: {len(train_dataset)}\tVal: {len(val_dataset)}\tTest: {len(test_dataset)}')

Sample size:
Train: 158	Val: 42	Test: 50


In [4]:
# infinite sampler for training
# not sure if cha nge shuffle to TRUE? (*)
# train_sampler = torch.utils.data.sampler.RandomSampler(train_dataset, replacement=True)
train_sampler = get_sampler(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_fn_with_padding, sampler=train_sampler, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_fn_with_padding, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_fn_with_padding, pin_memory=True)

In [5]:
model_longnet = giga_slide_enc(path='model/pub/slide_encoder.pth', global_pool=True)
# model_longnet = longn.glio_create_model(local_path='', model_arch="gigapath_slide_enc3l32d", in_chans=1536, global_pool=True)
model = SeriesHead(LongNetModel=model_longnet,
                   TaskHead=TaskHead(768 * len(feat_layers), num_classes),
                   feat_layers=feat_layers)

/mnt/f/workspace/pathology/gigapath
dilated_ratio:  [1, 2, 4, 8, 16]
segment_length:  [1024, 5792, 32768, 185363, 1048576]
Number of trainable LongNet parameters:  85148160
Global Pooling: True
 Successfully Loaded Pretrained GigaPath model from model/pub/slide_encoder.pth 
Slide encoder param # 86330880


In [6]:
# Train the model
pred_gather, target_gather = train(model, train_loader, val_loader, test_loader, **params)

Starting with ALL LongNet layers frozen

Progressive Unfreezing Configuration:
  Total LongNet layers: 12 (layers 0-11)
  Permanently frozen layers: 0--1 (first 0 layers)
  Trainable layer range: 0-11
  Initial active layers: None (all frozen)
  Unfreeze schedule: {10: [-1], 20: [-2], 30: [-3]}
  Auto-unfreeze head components: False


DETAILED PARAMETER BREAKDOWN

📍 Embedding Components:
  patch_embed         :          0/ 1,180,416 params  ✗ FROZEN
  cls_token           :          0/       768 params  ✗ FROZEN

📍 Encoder Layers:
  Layer  0:          0/ 7,095,552 params  ✗ FROZEN (can be unfrozen)
  Layer  1:          0/ 7,095,552 params  ✗ FROZEN (can be unfrozen)
  Layer  2:          0/ 7,095,552 params  ✗ FROZEN (can be unfrozen)
  Layer  3:          0/ 7,095,552 params  ✗ FROZEN (can be unfrozen)
  Layer  4:          0/ 7,095,552 params  ✗ FROZEN (can be unfrozen)
  Layer  5:          0/ 7,095,552 params  ✗ FROZEN (can be unfrozen)
  Layer  6:          0/ 7,095,552 params  ✗ FROZEN

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 0:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch [1/50]	Train Loss: 0.7167	LRs: TaskHead: 0.000280


Epoch 1:   0%|          | 0/40 [00:00<?, ?it/s]

...

Epoch [48/50]	Train Loss: 0.6755	LRs: TaskHead: 0.000139, LongNet: 0.000003


Epoch 48:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch [49/50]	Train Loss: 0.6799	LRs: TaskHead: 0.000110, LongNet: 0.000002


Epoch 49:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch [50/50]	Train Loss: 0.6742	LRs: TaskHead: 0.000100, LongNet: 0.000002
Evaluating at epoch 50...
Val Epoch [50/50] Acc: 0.488 F1: 0.030 Prec: 0.083 Rec: 0.019 AUROC: 0.527 AUPRC: 0.566
Loading best model with f1: 0.075
Evaluating on test set...
Test Acc: 0.507 F1: 0.083 Prec: 0.080 Rec: 0.086 AUROC: 0.443 AUPRC: 0.484
